In [4]:
def make_stc(subject, subjects_dir):
    
    import os
    import mne
    import mne_bids
    
    method = 'sLORETA'
    lambda2 = 1. / 3 ** 2
    spacing = 'oct6'
    
    if subject in subjects_names:
        description='cleaned+crop'
    else:
        description='cleaned'
    
    spacing = 'oct6'
    
    epo_bids_path = mne_bids.BIDSPath(subject=subject,
                         root=bids_derivatives,
                         task='vowels',
                         datatype='meg',
                         processing='tsss+mc+transtooptimal+0110hz+ssp',
                         description=description,
                         extension='.pos')
    
    epochs = mne.read_epochs(str(epo_bids_path.fpath) + '-epo.fif')
    
    fwd = mne.read_forward_solution(bids_derivatives + 'sub-' + subject + '/'
                               + subject + '_forward_solution_' + spacing + '-fwd.fif')
    
    inv = mne.minimum_norm.read_inverse_operator(bids_derivatives + 'sub-' + subject + '/'
                               + subject + '_inverse_operator_' + spacing + '-inv.fif')
    
    src_to = mne.setup_source_space(subject='fsaverage', spacing='oct6', subjects_dir=subjects_dir, add_dist=False)
    
    fwd = mne.read_forward_solution(bids_derivatives + 'sub-' + subject + '/'
                               + subject + '_forward_solution_' + spacing + '-fwd.fif')
    
    morph = mne.compute_source_morph(src=fwd["src"],
                                     src_to=src_to,
                                     subject_from='sub-' + subject,
                                     subject_to='fsaverage',
                                     subjects_dir=subjects_dir)
    

    os.makedirs(bids_derivatives + 'sub-' + subject + '/source_estimate/', exist_ok=True)
    
    for cond in ['dv','rv','mp','mr']:
        
        evoked = epochs[cond].average()
        
        stc = mne.minimum_norm.apply_inverse(evoked, inverse_operator=inv, method=method, lambda2=lambda2, pick_ori=None)
        
        stc.save(bids_derivatives + 'sub-' + subject + '/source_estimate/' +
                     subject + '_' + cond + '_stc', overwrite=True)
        
        stc = morph.apply(stc)
        
        stc.save(bids_derivatives + 'sub-' + subject + '/source_estimate/' + 
                     subject + '_' + cond + '_stc_morph', overwrite=True)
       
        del evoked, stc
    
    del inv, morph, fwd, epochs

def check_stc(subject, subjects_dir):
    
    import mne
    import matplotlib.pyplot as plt
    
    mne.viz.set_3d_options(antialias=False)
    
    f, axes = plt.subplots(1, 2, figsize=(15, 8))
    f.suptitle(subject) 
    
    cond = 'dv'
    
    for n_morph, (morph, subject_stc) in enumerate(zip(['', '_morph'],['sub-' + subject, 'fsaverage'])):
        
        stc = mne.read_source_estimate(bids_derivatives + 'sub-' + subject + '/source_estimate/' + 
                     subject + '_' + cond + '_stc' + morph)
        
        data = stc.copy().data
        for i in range(1201):
            stc.data[:,i] = abs(data[:,400:1000]).mean(1)
        
        brain = stc.plot(subject=subject_stc,
                            hemi = 'split',
                            views = 'lat',
                            initial_time = .3,
                            time_viewer = True,
                            show_traces = True,
                            surface = 'inflated',
                            size=(1000,500),
                            subjects_dir=subjects_dir,
                            background='white')
        
        screenshot = brain.screenshot()
        brain.close()
        axes[n_morph].imshow(screenshot)
        axes[n_morph].axis('off')
        axes[n_morph].set_title(cond + morph)
         
    f.tight_layout()
    plt.close()
    
    f.savefig(bids_derivatives + 'sub-' + subject + '/source_estimate/' +
              subject + '_check_stc.png')

    
subjects_names = ['Z201']

bids_root = r'D:\\ds005234\\'
bids_derivatives = bids_root + r'derivatives\\'

subjects_dir = r'D:\ds005234\derivatives\freesurfer'


for subject in subjects_names:
    make_stc(subject, subjects_dir)

for subject in subjects_names:
    check_stc(subject, subjects_dir)

Reading D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp_desc-cleaned+crop-epo.fif ...
    Read a total of 4 projection items:
        ECG-planar--0.200-0.400-PCA-01 (1 x 204) active
        ECG-planar--0.200-0.400-PCA-02 (1 x 204) active
        ECG-axial--0.200-0.400-PCA-01 (1 x 102) active
        ECG-axial--0.200-0.400-PCA-02 (1 x 102) active
    Found the data of interest:
        t =    -200.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1009 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Reading forward solution from D:\ds005234\derivatives\sub-Z201\Z201_forward_solution_oct6-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read MEG forward solution (81